# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 41.309 MiB


In [2]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [2] used 39.520 MiB RAM in 0.031s, peaked 0.000 MiB above current, total RAM usage 80.828 MiB


In [3]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=39.51953125, time_delta=0.030761241912841797, memory_peak=0, memory_usage=80.828125)

In [3] used 2.000 MiB RAM in 0.012s, peaked 0.000 MiB above current, total RAM usage 82.828 MiB


In [4]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

39.51953125

In [4] used -1.828 MiB RAM in 0.002s, peaked 1.828 MiB above current, total RAM usage 81.000 MiB


In [5]:
# This takes between 32 ~ 40 bytes per element:
meas.memory_delta * (2**20) / 1e6

41.439232

In [5] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 81.000 MiB


In [6]:
# What are these elements made of?
type(a[0])

int

In [6] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 81.000 MiB


In [7]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [7] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 81.000 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~40.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components.

![List diagram](http://www.brpreiss.com/books/opus7/html/img579.gif)

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [8]:
%load_ext memory_profiler

In [8] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 81.000 MiB


In [9]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 155.17 MiB, increment: 74.11 MiB
In [9] used 39.059 MiB RAM in 0.200s, peaked 0.000 MiB above current, total RAM usage 120.059 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## %time and %timeit

In [10]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 9.5 ms
In [10] used 0.012 MiB RAM in 0.011s, peaked 0.000 MiB above current, total RAM usage 120.070 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 1~5 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [11]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

100 loops, best of 3: 6.55 ms per loop
In [11] used 0.016 MiB RAM in 2.734s, peaked 0.000 MiB above current, total RAM usage 120.086 MiB


Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [12]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

100 loops, best of 3: 6.54 ms per loop
[0.653744095005095, 0.6584454510593787, 0.6539616700029001]
0.00653744095005095
In [12] used 0.000 MiB RAM in 2.697s, peaked 0.000 MiB above current, total RAM usage 120.086 MiB


And one can specify the number of loops (-n) and the number of repetitions (-r):

In [13]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

1 loop, best of 1: 6.71 ms per loop
[0.006711989990435541]
0.006711989990435541
In [13] used 0.000 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 120.086 MiB


### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [14]:
d = dict(("key: %i"%i, i*2) for i in a)

In [14] used 151.703 MiB RAM in 0.421s, peaked 0.000 MiB above current, total RAM usage 271.789 MiB


In [15]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

88

In [15] used 0.215 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 272.004 MiB


So, `sys.getsizeof()` is telling us that the keys + values take ~90 MB.  However, our `MemWatcher` instance is reporting just ~150 MB, so that means that the dict structure takes around ~60 MB.  The take away lesson is: do not assume that the consumption is just in the elements; the data container can also take quite a bit of memory!